_____

<table align="left" width=100%>
    <td>
        <div style="text-align: center;">
          <img src="./images/bar.png" alt="entidades financiadoras"/>
        </div>
    </td>
    <td>
        <p style="text-align: center; font-size:24px;"><b>Introduction to Data Science</b></p>
        <p style="text-align: center; font-size:18px;"><b>Master in Electrical and Computer Engineering</b></p>
        <p style="text-align: center; font-size:14px;"><b>Pedro Cardoso (pcardoso@ualg.pt)</b></p>
    </td>
</table>

_____

__Short Lesson Title:__ Pandas Series: The Building Blocks

*__Summary:__ This lesson introduces Pandas Series, the foundational one-dimensional labeled array data structure in Python's Pandas library. Students will learn how to create Series from various data types, including NumPy arrays, Python dictionaries, and scalar values. The lesson covers essential Series attributes like index, values, and dtype, and demonstrates how to access and modify data using label-based indexing (loc), position-based indexing (iloc), and boolean indexing. It also explores how to perform vectorized operations on Series, handling unaligned data and missing values (NaN). Additionally, the lesson covers how to describe and visualize data using methods like describe, plot, hist, box, and kde. Finally, it explains the performance implications of using unique vs. non-unique indexes. Through practical examples, students will gain a solid understanding of Series and their role in data analysis.*

# Series

A Pandas Series is like a column in a table: it is a one-dimensional array holding data of any type. Furthermore, it has an associated array of data labels, called its index.

A Pandas Series can be created from various data sources, including:
- A __Python dictionary__, where keys serve as the index and values as the data.
- __NumPy array__ (ndarray), with optional custom indexing.
- __Scalar value__, which is broadcasted across all index entries.

So, in essence
- **Series is a one-dimensional labeled array** capable of holding any data type (integers, strings, floating point numbers, Python objects, etc.). 
- The **axis labels** are collectively referred to as the index. 



![images/01_table_series.svg](images/01_table_series.svg)


Let us start by importing the necessary libraries:
- pandas - the main library for data manipulation and analysis
- numpy - the main library for numerical computing
- matplotlib - a 2D plotting library which produces publication quality figures

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## Creating a Series

To create a `Series` you can use the `Series` constructor:

```s = pd.Series(data, index=index)```

where `data` can be many different things.

If `data` is not given, it defaults to an empty Series.

In [ ]:
s = pd.Series(dtype='float')
s


### From ` ndarray `
- If data is an `ndarray`, index must be the same length as data. 
- If no index is passed, one will be created having values [0, ..., len(data) - 1].

First, let us create a numpy array with 5 random numbers

In [ ]:
v = np.random.normal(size=5)
print(type(v))
v

Now, the series can be created from the numpy array defined above, using an index passed directly as a parameter

In [ ]:
s = pd.Series(v, 
              index=tuple('abcde'))
s

### From dict

Series can also be instantiated from dictionaries.

When the data is a dict, and an index is not passed, the Series index will be ordered by the dict’s insertion order (Python >= 3.6 and Pandas >= 0.23).

In [ ]:
d = {k: v for k, v in zip('abcde', np.random.normal(size=5))}
d

In [ ]:
s = pd.Series(d)
s

### From Scalar

If data is a scalar value, an index must be provided. The value will be repeated to match the length of index.

In [ ]:
s = pd.Series(1.0, index=tuple('olivia'))
s

## Accessing data

Let us create a series to illustrate the access to the data

In [ ]:
s = pd.Series(v, index=tuple('abcde'))
s

The index is given by the `index` property

In [ ]:
s.index

The np.array of values is given by the `values` property

In [ ]:
s.values

And, **values can be accessed by label index** (like a dict) or by position index (like a list). 

For example, the dot notation can be used to access the values by label index

In [ ]:
s.a

or access using the index as a key

In [ ]:
s['a']

The `.loc` attribute can also be used to access the values by label index

In [ ]:
s.loc['a']

or 0-based index using the `.iloc` attribute

In [ ]:
s.iloc[0]

It is also possible to do **slicing** (We'll see more of it latter)

In [ ]:
s['b':'d']

But be careful, indexes are not necessarily unique.

In [ ]:
s_not_unique_idx = pd.Series(1.0, index=tuple('olivia'))
s_not_unique_idx

In [ ]:
s_not_unique_idx['i']

- when index is **unique**, pandas use a hashtable to map key to value: $\mathcal{O}(1)$
- when index is **non-unique and sorted**, pandas use binary search: $\mathcal{O}(\log N)$
- when index is **non-unique and random ordered**, pandas need to check all the keys in the index: $\mathcal{O}(N)$

Let us then check the access time for these cases (recall 	$ns= 10^{-9}, µs = 10^{-6}, ms = 10^{-3}$

In [ ]:
idx_unique = 'defghabcijkqrstuvwlmnopxyz'

print('index length', len(idx_unique), ' (unique: ', len(set(idx_unique)), ')')

# the values are not important, so we can use the same value for all
s = pd.Series(1.0, index=tuple(idx_unique))

print("Unique & unsorted")
%timeit s['z']

print("Unique & sorted")
s.sort_index(inplace=True)
%timeit s['z']

In [ ]:
idx_not_unique = "oliviaoliviaoliviaoliviaol"
print('index length', len(idx_not_unique), ' (unique: ', len(set(idx_not_unique)), ')')

s = pd.Series(1.0, index=tuple(idx_not_unique))

print("Not unique & unsorted")
%timeit s['a']

print("Not unique & sorted")
s.sort_index(inplace=True)
%timeit s['a']

In the above examples, note the use of the `inplace=True` parameter to modify the original Series. By default, the `inplace` parameter is `False` and a new Series is returned.

## NaN
`NaN` (*not a number*) is the standard missing data marker used in pandas. `NaN` is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation.



In [ ]:
d = {'a' : 1, 'b' : 2, 'c': 3}

pd.Series(d, index=tuple('abz'))

## Get series values types
The type of the values can be obtained via `dtype`.


In [ ]:
s

In [ ]:
# as of pandas 0.24.0, the default dtype is 'float64'
s.dtype

And of the values themselve

In [ ]:
# array of values as a numpy array 
print(type(s.values))

In [ ]:
# as a PandasArray
print(type(s.array))
s.array

## Slicing
Series acts very similarly to a ndarray. 
However, operations such as slicing will also slice the index. Let us start by creating a series

In [ ]:
s = pd.Series(np.random.normal(size=5), index=tuple('abcde'))
s

Get the indeces that satisfy some filter

In [ ]:
s > 0

Given the filter, you can get the values

In [ ]:
s[s>0]

Another example, now using the median value as a filter to get the values that are greater than the median

In [ ]:
s[s>s.median()]

To get values given the 0-indeces use `iloc` 

In [ ]:
s.iloc[[0, 1, 2]]

To get values given the label indeces use `loc` or key access with `[]`

In [ ]:
s[['a', 'c', 'e']]

## Modifying data

To change the series' values you can use the same indexing access

In [ ]:
s["a"] = 10
s

In [ ]:
s.loc["b"] = 20
s

In [ ]:
s.iloc[2:] = 30
s

In [ ]:
s.iloc[-1] = 40
s

## Exercise
Before running them, try to guess the output of the followiong lines 

In [ ]:
s1 = pd.Series(np.random.randn(10), index=list(range(0, 50, 5)))
s1

In [ ]:
s1[5]

In [ ]:
s1.loc[5]

In [ ]:
s1.loc[:5]

In [ ]:
# !
s1[:5]

In [ ]:
s1.iloc[:5]

In [ ]:
s1.iloc[5]

To avoid any unexpected behaviour, it is better to use `loc` and `iloc` to access the data.

## Operations
When working with raw NumPy arrays, looping through value-by-value is usually not necessary. The same is true when working with Series in pandas. Series can also be passed into most NumPy methods expecting an ndarray.

In [ ]:
r = pd.Series([1, 2, 3, 4, 5], index=tuple('abcde'))
s = pd.Series([1, 2, 3, 4, 5], index=tuple('edcba'))

print("---- r ----")
print(r)

print("---- s ----")
print(s)

In [ ]:
r + s

In [ ]:
2 * r

In [ ]:
r ** 2

In [ ]:
2 ** r

The result of an operation between unaligned Series will have the union of the indexes involved. If a label is not found in one Series or the other, the result will be marked as missing NaN. 

In [ ]:
s[1:] + s [:-1]

similar

In [ ]:
s.iloc[1:] + s.iloc[:-1]

## Describe & Visualize data

Let us create a series with 10000 random numbers that follow a normal distribution

In [ ]:
s = pd.Series(np.random.normal(size=10000))
s.head(10)

In [ ]:
s.info()

In [ ]:
s.describe()

You can do several kinds of plot such as line, bar, box, etc. (see https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.plot.html)

In [ ]:
s.plot(figsize=(25, 10))
plt.show()

In [ ]:
s.plot(kind='box')
plt.show()

In [ ]:
s.plot(kind='hist', bins=50)
plt.show()

In [ ]:
s.plot(kind='hist', bins=50, cumulative=True)
plt.show()

In [ ]:
s.plot(kind='kde')
plt.show()